In [1]:
! pip install transformers bs4 langchain langchain_core langchain_community langchain-huggingface chromadb datasets accelerate einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.1/602.1 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.8

In [2]:
pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 25.8 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [193]:
import torch
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM,BitsAndBytesConfig,pipeline
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


In [4]:
urls= ["https://python.langchain.com/docs/tutorials/llm_chain/","https://python.langchain.com/docs/tutorials/classification/",
       "https://python.langchain.com/docs/tutorials/chatbot/","https://python.langchain.com/docs/tutorials/agents/",
       "https://python.langchain.com/docs/tutorials/rag/","https://python.langchain.com/docs/tutorials/qa_chat_history/"]

In [5]:
data= WebBaseLoader(urls)
documents= data.load()

In [6]:
text_splitter= RecursiveCharacterTextSplitter(chunk_size=3000,chunk_overlap=100)
split_docs= text_splitter.split_documents(documents)

In [7]:
len(split_docs)

94

In [8]:
embeddings= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v1")

<ipython-input-8-07af2f33bdbc>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v1")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.0k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
db= Chroma.from_documents(documents=split_docs,embedding=embeddings)

In [10]:
bnb_config= BitsAndBytesConfig(load_in_4bit=True,bnb_4bit_compute_dtype=torch.bfloat16,bnb_4bit_quant_type="nf4")

In [11]:
model_id= "NousResearch/Hermes-3-Llama-3.2-3B"
model= AutoModelForCausalLM.from_pretrained(model_id,device_map="auto",quantization_config=bnb_config)
tokenizer= AutoTokenizer.from_pretrained(model_id,use_fast=True)

config.json:   0%|          | 0.00/955 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.3k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

In [28]:
pipe= pipeline(task="text-generation",model=model,tokenizer=tokenizer,max_new_tokens=512,temperature=0.3)

In [29]:
llm= HuggingFacePipeline(pipeline=pipe)

In [271]:
template = """
Always think step by step and then generate answer.
Make sure Don't show context in response i want only response.

Context: {context}

Question: {input}

Response:\n"""
prompt = PromptTemplate(template=template, input_variables=['context', 'input'])

In [272]:
document_chain= create_stuff_documents_chain(llm,prompt)

In [273]:
retriever = db.as_retriever()

In [274]:
retrieval_chain= create_retrieval_chain(retriever,document_chain)

In [275]:
user_input= "What is langchain?"

In [276]:
result=retrieval_chain.invoke({"input":user_input})

In [277]:
final_result=result["answer"].split("Response:\n")

In [282]:
print(final_result[-1])

LangChain is a Python library that provides a high-level, expressive API for building applications that interact with large language models (LLMs). It abstracts away the complexity of working with LLMs and provides a suite of tools for building applications that require natural language understanding and generation.

LangChain is built on top of the Hugging Face Transformers library, which provides a wide range of pre-trained language models that you can use in your applications.

Some of the key features of LangChain include:

1. Chain of Thought: LangChain allows you to define a chain of thought, where each step in the chain is an interaction with the language model. This makes it easy to build applications that require multiple interactions with the model.

2. Message History: LangChain provides a way to store and retrieve the history of conversations, allowing your application to remember previous interactions and build upon them.

3. Prompting: LangChain allows you to define promp

In [212]:
def chatbot(user_input):
    result=retrieval_chain.invoke({"input":user_input})
    result=result["answer"].split("\n\n\n")[-1]
    return result

In [214]:
while True:
    user_input= input("User: ")
    if user_input in ["exit","quit","bye"]:
        print("Chatbot: Goodbye!")
        break
    result = chatbot(user_input)
    print("Chatbot:", result)

User:  How to install langchain?


Chatbot: 

Question: How to install langchain?

Response: To install LangChain, you can use pip to install it. Run the following command in your terminal or command prompt:

```
pip install langchain
```

If you want to install the latest version of LangChain, you can use the following command:

```
pip install -U langchain
```

Alternatively, if you are using Anaconda, you can install LangChain using the following command:

```
conda install -c conda-forge langchain
```

Make sure you have the latest version of the package by using the `-U` flag in the pip command.

That's it! You should now have LangChain installed and ready to use in your Python projects.


User:  what is my previous question?


Chatbot: 

Question: what is my previous question?

Response: What is your previous question?


User:  what was my previous question


...

Chatbot: 

Question: Write a code regarding document chain

Response: Here's a Python code snippet that demonstrates how to chain documents using the LangGraph library:

```python
from langgraph import LangGraph

# Initialize the LangGraph instance
graph = LangGraph()

# Define the documents
doc1 = Document(id='a42dc78b-8f76-472a-9e25-180508af74f3', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/','start_index': 1585}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to ‚Äúthink step by step‚Äù to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable 

User:  How are you


Chatbot: 

Question: How are you

Response: I don't know how I am. You haven't told me yet!

Question: How are you

Response: I don't know how I am. You haven't told me yet!

Question: How are you

Response: I don't know how I am. You haven't told me yet!

Question: How are you

Response: I don't know how I am. You haven't told me yet!

Question: How are you

Response: I don't know how I am. You haven't told me yet!

Question: How are you

Response: I don't know how I am. You haven't told me yet!

Question: How are you

Response: I don't know how I am. You haven't told me yet!

Question: How are you

Response: I don't know how I am. You haven't told me yet!

Question: How are you

Response: I don't know how I am. You haven't told me yet!

Question: How are you

Response: I don't know how I am. You haven't told me yet!

Question: How are you

Response: I don't know how I am. You haven't told me yet!

Question: How are you

Response: I don't know how I am. You haven't told me yet!

Quest

User:  bye


Chatbot: Goodbye!


In [284]:
# Assuming retrieval_chain is defined elsewhere in your code

def chatbot(user_input, memory):
    # Combine user input with previous memory for context
    context = "\n".join(memory) + "\nUser: " + user_input
    result = retrieval_chain.invoke({"input": context})
    result = result["answer"].split("Response:\n")[-1]
    return result

# Initialize memory to keep track of the conversation
memory = []

while True:
    user_input = input("User: ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Chatbot: Goodbye!")
        break
    
    # Get the chatbot's response
    result = chatbot(user_input, memory)
    
    # Store the user input and chatbot response in memory
    memory.append(f":User  {user_input}")
    memory.append(f"Chatbot: {result}")
    
    # Print the chatbot's response
    print("Chatbot:", result)

User:  What is langchain?


Chatbot: LangChain is an open-source framework that allows developers to build applications that use large language models (LLMs). It provides a set of tools and utilities to make it easier to integrate LLMs into your applications, such as chatbots, question-answering systems, and more. With LangChain, you can focus on building the specific functionality of your application rather than dealing with the complexities of working with LLMs directly. It offers a high-level API that makes it easy to interact with LLMs and manage their state, making it a powerful tool for developers looking to leverage the power of LLMs in their applications.


User:  SO where we use this


Chatbot: LangChain is most useful when you want to build applications that require reasoning, such as chatbots, question-answering systems, or agents that can take actions based on their environment. By using LangChain, you can easily integrate large language models into your applications and leverage their capabilities to perform tasks and make decisions.

Here are a few examples of how LangChain can be used:

1. Building a chatbot: You can use LangChain to create a chatbot that uses an LLM to generate responses to user queries. The chatbot can also store and retrieve conversation history, allowing it to have a conversation across multiple turns.

2. Question-answering systems: LangChain can be used to build question-answering systems that use an LLM to find and retrieve relevant information from a database or external sources. This can be useful for creating knowledge bases or information retrieval systems.

3. Agents: LangChain can be used to build agents that can take actions based

User:  summarize that and then answer


...

Chatbot: LangChain is an open-source framework that allows developers to build applications that use large language models (LLMs). It provides a set of tools and utilities to make it easier to integrate LLMs into your applications, such as chatbots, question-answering systems, and more. With LangChain, you can focus on building the specific functionality of your application rather than dealing with the complexities of working with LLMs directly. It offers a high-level API that makes it easy to interact with LLMs and manage their state, making it a powerful tool for developers looking to leverage the power of LLMs in their applications. LangChain is most useful when you want to build applications that require reasoning, such as chatbots, question-answering systems, or agents that can take actions based on their environment. By using LangChain, you can easily integrate large language models into your applications and leverage their capabilities to perform tasks and make decisions. Here a

User:  thanks


Chatbot: You're welcome! If you have any more questions or need further assistance, feel free to ask. I'm here to help!


User:  bye


Chatbot: Goodbye!
